In [2]:
import numpy as np
import pandas as pd
import redis

In [28]:
#redis-14584.c1.asia-northeast1-1.gce.redns.redis-cloud.com:14584
#dcZEr1OivCI6rRb27Eqw8mdCC9Lez0wJ
hostname = 'redis-14584.c1.asia-northeast1-1.gce.redns.redis-cloud.com'
port = 14584
password = 'dcZEr1OivCI6rRb27Eqw8mdCC9Lez0wJ'

r = redis.Redis(host=hostname, port=port, password=password)

In [29]:
r.ping()

True

In [5]:
r.keys()

[b'MESSAGE', b'foo', b'mylist', b'KEY']

In [9]:
np_file = np.load('dataframe_students_teacher.npz', allow_pickle=True)
x_values = np_file['arr_0']
col_names  = np_file['arr_1']

df = pd.DataFrame(x_values, columns=col_names)
df

,Name,Role,Facial_Features
0,Angelina Jolie,Student,"[0.4324205, 1.0748497, -0.128624, -1.8332914, ..."
1,Barack Obama,Teacher,"[0.9485806, -0.5584053, 0.094820514, -1.279659..."
2,Chris Evans,Student,"[0.72017133, -1.468415, -0.7781255, -1.4601074..."
3,Morgan Freeman,Teacher,"[0.4853508, -0.61925495, -0.4069931, 1.0687788..."
4,Scarlett Johansson,Student,"[0.19131932, -0.4884374, -1.6902335, 0.4423968..."


In [10]:
df['name_role'] = df['Name'] + '@' + df['Role']
df

,Name,Role,Facial_Features,name_role
0,Angelina Jolie,Student,"[0.4324205, 1.0748497, -0.128624, -1.8332914, ...",Angelina Jolie@Student
1,Barack Obama,Teacher,"[0.9485806, -0.5584053, 0.094820514, -1.279659...",Barack Obama@Teacher
2,Chris Evans,Student,"[0.72017133, -1.468415, -0.7781255, -1.4601074...",Chris Evans@Student
3,Morgan Freeman,Teacher,"[0.4853508, -0.61925495, -0.4069931, 1.0687788...",Morgan Freeman@Teacher
4,Scarlett Johansson,Student,"[0.19131932, -0.4884374, -1.6902335, 0.4423968...",Scarlett Johansson@Student


In [11]:
records = df[ ['name_role', 'Facial_Features' ]].to_dict(orient='records' )
len(records)

5

In [13]:
for record in records:
    name_role = record['name_role']
    vector = record['Facial_Features']
    #convert numpy array to byte
    byte_vector = vector.tobytes()
    #store the byte vector in redis
    r.hset(name='academy:register', key=name_role, value=byte_vector)

Registration form

In [14]:
import cv2
from insightface.app import FaceAnalysis

In [15]:
faceapp = FaceAnalysis(name='buffalo_sc')
faceapp.prepare(ctx_id=0, det_thresh=0.5, det_size=(640,640))

Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\Users\tuand/.insightface\models\buffalo_sc\det_500m.onnx detection [1, 3, '?', '?'] 127.5 128.0
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
find model: C:\Users\tuand/.insightface\models\buffalo_sc\w600k_mbf.onnx recognition ['None', 3, 112, 112] 127.5 127.5
set det-size: (640, 640)


D:\Code\Attendance_System\attendance_system\Lib\site-packages\onnxruntime\capi\onnxruntime_inference_collection.py:69: UserWarning: Specified provider 'CUDAExecutionProvider' is not in available provider names.Available providers: 'AzureExecutionProvider, CPUExecutionProvider'
  warnings.warn(


In [19]:
#input
person_name = input('Enter the name of the person: ')
trials = 3
for i in range(trials):
    role = input("""
    Please choose the role of the person:
    1. Student
    2. Teacher
    """)
    if role in ['1', '2']:
        if role == '1':
            role = 'Student'
        else:
            role = 'Teacher'
        break
    else:
        print('Invalid choice. Please try again')
        if i == trials - 1:
            print('Maximum number of trials reached')

key = person_name + '@' + role
print('Name:', person_name)
print('Role:', role)
print('Key:', key)

Name: abc
Role: Student
Key: abc@Student


In [23]:
cap = cv2.VideoCapture(0) # 0 default camera, 1 external camera
face_embeddings = []
sample = 0
while True:
    ret, frame = cap.read()
    if ret == False:
        print('Unable to read camera')
        break
    # get results from insightface model
    results = faceapp.get(frame,max_num=1)
    for res in results:
        sample += 1
        x1, y1, x2,y2=res['bbox'].astype(int)
        cv2.rectangle(frame, (x1,y1),(x2,y2),(0,255,0),1)

        # facial features
        embeddings = res['embedding' ]
        face_embeddings.append(embeddings)

    if sample >=  200:
        break
    cv2.imshow('frame', frame)
    if cv2.waitKey(1) == ord('q'): # this is trigger only when I press Letter q (Lowercase q) in my keyboard
        break

cap.release()
cv2.destroyAllWindows ()


In [24]:
len(face_embeddings)

200

In [25]:
x_mean = np.asarray(face_embeddings).mean(axis=0)
x_mean.shape

(512,)

In [26]:
#convert numpy array to byte
x_mean_bytes = x_mean.tobytes()


In [30]:
#store the key and the byte vector in redis
r.hset(name='academy:register', key=key, value=x_mean_bytes)

1